# Learning and Decision Making

## Laboratory 6: Reinforcement learning

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. The windy gridworld domain

Consider the larger version of the windy gridworld domain depicted in the figure below.

<img src="windy.png" width="400px">

In it, a boat must navigate a 7 &times; 10 gridworld, to reach the goal cell, marked with _G_. There is a crosswind upward through the middle of the grid, in the direction indicated by the gray arrows. The boat has available the standard four actions -- _Up_, _Down_, _Left_ and _Right_. In the region affected by the wind, however, the resulting next state is shifted upward as a consequence of the crosswind, the strength of which varies from column to column. The strength of the wind is given below each column, and corresponds to the number of cells that the movement is shifted upward. For example, if the boat is one cell to the right of the goal, then the action _Left_ takes you to the cell just above the goal.

The agent pays a cost of 1 in every step before reaching the goal. The problem can be described as an MDP $(\mathcal{X},\mathcal{A},\mathbf{P},c,\gamma)$ as follows.

In [2]:
%matplotlib notebook
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

np.set_printoptions(threshold=10)

# Problem specific parameters
WIND = (0, 0, 0, 1, 1, 1, 2, 2, 1, 0)
nrows = 7
ncols = 10
init = [3, 0]
goal = [3, 7]

# States
X = [[x, y] for x in range(nrows) for y in range(ncols)]
nX = len(X)

# Actions
A = ['U', 'D', 'L', 'R']
nA = len(A)

# Transition probabilities
P = dict()
P['U'] = np.zeros((nX, nX))
P['D'] = np.zeros((nX, nX))
P['L'] = np.zeros((nX, nX))
P['R'] = np.zeros((nX, nX))

for i in range(len(X)):
    x = X[i]
    y = dict()
    
    y['U'] = [x[0] - WIND[x[1]] - 1, x[1]]
    y['D'] = [x[0] - WIND[x[1]] + 1, x[1]]
    y['L'] = [x[0] - WIND[x[1]], x[1] - 1]
    y['R'] = [x[0] - WIND[x[1]], x[1] + 1]
    
    for k in y:
        y[k][0] = max(min(y[k][0], nrows - 1), 0)
        y[k][1] = max(min(y[k][1], ncols - 1), 0)
        j = X.index(y[k])
        P[k][i, j] = 1

c = np.ones((nX, nA))
c[X.index(goal), :] = 0

gamma = 0.99

# -- Pretty print

print('\n- MDP problem specification: -\n')

print('States:')
print(np.array(X))P[i]

print('\nActions:')
print(A)

print('\nTransition probabilities:')
for a in A:
    print('Action', a)
    print(P[a])
    
print('\ncost:')
print(c)

print('\nStart state:', init)
print('\nGoal state:', goal)


- MDP problem specification: -

States:
[[0 0]
 [0 1]
 [0 2]
 ..., 
 [6 7]
 [6 8]
 [6 9]]

Actions:
['U', 'D', 'L', 'R']

Transition probabilities:
Action U
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Action D
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
Action L
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
Action R
[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]

cost:
[[ 1.  1

---

#### Activity 1.        

Compute the optimal _Q_-function for the MDP defined above using value iteration. As your stopping condition, use an error between iterations smaller than `1e-8`.

---

In [40]:
def calculateQ(q,P=P,c=c):
    j = q.min(axis=1).reshape(-1, 1)
    result = np.zeros(shape=(nX, nA))
    for i in range(nA):
        result[:, i] = (c[:, i].reshape(-1, 1) + gamma * P[A[i]].dot(j)).reshape(-1)
    return result

Q = np.zeros((nX, nA))
oldQ = Q
err=1
iterations=0
while err>1e-8:
    oldQ = Q
    Q = calculateQ(oldQ)
    err = np.linalg.norm(Q - oldQ)
    iterations+=1
print(Q)
print('iterations = %d' % iterations)

[[ 88.97864983  88.97864983  88.97864983  88.86732306]
 [ 88.86732306  88.86732306  88.97864983  88.75487178]
 [ 88.75487178  88.75487178  88.86732306  88.64128462]
 ..., 
 [ 87.18534608  87.44035769  88.29359262  87.31349262]
 [ 87.31349262  87.44035769  87.44035769  87.56595412]
 [ 87.56595412  87.56595412  87.44035769  87.56595412]]
iterations = 2108


---

#### Activity 2.        

Write down a Python function that, given a Q-function $Q$ and a state $x$, selects a random action using the $\epsilon$-greedy policy obtained from $Q$ for state $x$. Your function should receive an optional parameter, corresponding to $\epsilon$, with default value of 0.1. 

**Note:** In the case of two actions with the same value, your $\epsilon$-greedy policy should randomize between the two.

---

In [42]:
import random
def e_greedy(Q, x, eps=0.1):
    if random.random() < eps:
        return random.randint(0, nA-1)
    else:
        return random.choice(np.where(np.isclose(Q[x],Q[x].min())))[0]



### 2. Model-based learning

You will now run the model-based learning algorithm discussed in class, and evaluate its learning performance.

---

#### Activity 3.        

Run the model-based reinforcement learning algorithm discussed in class to compute $Q^*$ for 100,000 iterations. Initialize each transition probability matrix as the identity and the cost function as all-zeros. Use an $\epsilon$-greedy policy with $\epsilon=0.1$ (use the function from Activity 2). Note that, at each step,

* You will need to select an action according to the $\epsilon$-greedy policy;
* The state and action, you will then compute the cost and generate the next state; 
* With this transition information (state, action, cost, next-state), you can now perform an update. 
* When updating the components $(x,a)$ of the model, use the step-size

$$\alpha_t=\frac{1}{N_t(x,a)+1},$$

where $N_t(x,a)$ is the number of visits to the pair $(x,a)$ up to time step $t$.

In order to ensure that your algorithm visits every state and action a sufficient number of times, after the boat reaches the goal cell, make one further step, the corresponding update, and then reset the position of the boat to a random state in the environment.

Plot the norm $\|Q^*-Q^{(k)}\|$ every 500 iterations of your method, where $Q^*$ is the optimal _Q_~function computed in Activity 1.

**Note:** The simulation may take a bit. Don't despair.

---

In [46]:
N = np.zeros(shape=(nX, nA))
P_estimated = dict()
P_estimated['U'] = np.identity(nX)
P_estimated['D'] = np.identity(nX)
P_estimated['L'] = np.identity(nX)
P_estimated['R'] = np.identity(nX)
c_estimated = np.zeros((nX, nA))
Q_estimated = np.zeros((nX, nA))
x0 = X.index(init)

def simulate(x, a):
    newx = np.random.choice(np.arange(nX), p=P[A[a]][x])
    cost = c[x,a]
    return newx, cost

currentx = x0
i=0
while i < 100000:
    action = e_greedy(Q_estimated, currentx)
    N[currentx, action] += 1
    newx, cost = simulate(currentx, action)
    alpha = 1./(N[currentx,action] +1)
    II = np.zeros((1, nX))
    II[0,newx] = 1
    P_estimated[A[action]][currentx,None] += alpha*(II - P_estimated[A[action]][currentx,None])
    c_estimated[currentx, action] += alpha*(cost - c_estimated[currentx, action])
    
    if currentx == X.index(goal):
        currentx = x0
    else:
        currentx = newx
        
    Q_estimated = calculateQ(Q_estimated, P=P_estimated, c=c_estimated)
    
    i+=1
    if (i%500==0):
        print(np.linalg.norm(Q_estimated-Q))
    

1407.40383431
912.551248281
519.530313218
483.089152709
478.674173526
477.294479763
476.597926695
476.167977585
475.882011714
475.686977271
475.538950265
475.411366627
471.650946434
471.563232291
471.502882261
471.458028979
471.420222788
471.383891008
471.354792224
471.328065941
471.300201795
471.279591968
471.248888064
471.230743557
471.212964806
471.198192008
471.183909366
471.171337924
471.160186245
471.147427377
471.135815738
471.12471888
471.110556181
471.100724007
471.093037425
471.086564451
471.07391104
471.066030447
471.060013125
471.054622717
471.048642853
471.043514834
471.038150709
471.031531715
471.027553693
471.02153221
471.016255248
471.012041248
471.008316678
471.004498566
470.997536424
470.99353741
470.987575279
470.984620999
470.981307047
470.978250693
470.974766229
470.97192931
470.966759637
470.964522855
470.961873025
470.959312208
470.955054516
470.953124961
470.949643889
470.947084019
470.942792585
470.939236371
470.936910789
470.935394456
470.933325869
470.9315885

In [31]:
print(X[simulate(X.index(init),3)[0]])

[3, 1]


### 3. Temporal-difference learning

You will now run both Q-learning and SARSA, and compare their learning performance with that of the model-based method just studied.

---

#### Activity 4.        

Repeat Activity 3 but using the _Q_-learning algorithm with a learning rate $\alpha=0.3$.

---

In [48]:
Q_estimated = np.zeros((nX, nA))
x0 = X.index(init)
learning_rate=0.3

currentx = x0
i=0
while i < 100000:
    action = e_greedy(Q_estimated, currentx)
    newx, cost = simulate(currentx, action)

    Q_estimated[currentx, action] += learning_rate*(cost + gamma*Q_estimated[newx].min() -  Q_estimated[currentx, action])
    
    if currentx == X.index(goal):
        currentx = x0
    else:
        currentx = newx
    
    i+=1
    if (i%500==0):
        print(np.linalg.norm(Q_estimated-Q))
    

1469.83328833
1461.33448022
1452.88898352
1444.60280786
1436.32141539
1428.20594222
1420.00048644
1411.78087389
1403.57180785
1395.52306263
1387.34205056
1379.29048512
1371.31379497
1363.24611292
1355.47251133
1347.85032002
1340.07421297
1332.35039065
1324.61743689
1317.02987951
1309.50531874
1301.93287202
1294.49490297
1287.09831044
1279.83695551
1272.454507
1265.11272057
1258.03229589
1250.83562819
1243.80105192
1236.68118071
1229.60743197
1222.71156593
1215.80723453
1208.88069507
1202.03047724
1195.23798562
1188.53195277
1181.85287964
1175.16761834
1168.52527818
1161.91809449
1155.47573813
1149.00636623
1142.62618206
1136.25089564
1129.88834837
1123.66851741
1117.3962942
1111.20794134
1105.05979545
1099.07051575
1092.98365138
1086.8675894
1080.86563247
1074.92794944
1069.07997002
1063.20733216
1057.43730074
1051.67681309
1045.87567365
1040.15760825
1034.49037514
1028.79146381
1023.28026994
1017.70198752
1012.16866715
1006.74102243
1001.29493105
995.918470817
990.639406953
985.356112

---

#### Activity 5.

Repeat Activity 4 but using the SARSA algorithm.

---

In [5]:
Q_estimated = np.zeros((nX, nA))
x0 = X.index(init)
learning_rate=0.3

currentx = x0
nextaction = None
i=0
while i < 100000:
    action = e_greedy(Q_estimated, currentx)
    newx, cost = simulate(currentx, action)
    nextaction = e_greedy(Q_estimated, newx)
    Q_estimated[currentx, action] += learning_rate*(cost + gamma*Q_estimated[newx].min() -  Q_estimated[currentx, action])
    
    if currentx == X.index(goal):
        currentx = x0
    else:
        currentx = newx
    
    i+=1
    if (i%500==0):
        print(np.linalg.norm(Q_estimated-Q))
    

---

#### Activity 6.

Discuss the differences observed between the performance of the three methods.

---

_Add your discussion here._